Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

# Wiekszosc komorek nie bedzie mialo odpalonego display, bo nie moglem exportowac ;(

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/Files")

In [0]:
filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = spark.read.format("csv"). \
              option("header","true"). \
              option("inferSchema","true"). \
              load(filePath)

display(namesDf)

In [0]:
from pyspark.sql.functions import *
namesDf = namesDf.withColumn("current_epoch_timestamp", unix_timestamp(current_timestamp())).withColumn("height_feet", lit(round(namesDf["height"] / 30.48, 2)))

In [0]:
# najpopularniejsze imie
most_popular_filtered = namesDf.withColumn("first_name", split(namesDf["name"], " ")[0]).select("name", "first_name").groupBy("first_name").count().sort(desc("count")).limit(1)

display(most_popular_filtered)
# najpopularniejsze imie to John

In [0]:
most_popular_filtered.explain(mode='formatted')

In [0]:
# display(namesDf.explain(mode="formatted"))

In [0]:
namesDf = namesDf.withColumn("date_of_birth", to_date(namesDf["date_of_birth"], "dd.mm.yyyy")) \
                 .withColumn("date_of_death", to_date(namesDf["date_of_death"], "dd.mm.yyyy"))
namesDf.dtypes

In [0]:
namesDf = namesDf.withColumn("age", year("date_of_death") - year("date_of_birth"))

In [0]:
namesDf.explain()

In [0]:
namesDf = namesDf.drop("bio", "death_details")
# display(namesDf)

In [0]:
namesDf.explain(mode="formatted")

In [0]:
# camelCase
columns = [column.title().replace("_", "") for column in namesDf.columns]

for colOld, colNew in zip(namesDf.columns, columns):
    namesDf = namesDf.withColumnRenamed(colOld, colNew)

display(namesDf)

In [0]:
namesDf.explain()

In [0]:
namesDf = namesDf.sort(asc("Name"))

display(namesDf)

In [0]:
namesDf.explain()

Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
filePath = "dbfs:/FileStore/tables/Files/movies.csv"
moviesDf = spark.read.format("csv"). \
                option("header","true"). \
                option("inferSchema","true"). \
                load(filePath)

display(moviesDf)

In [0]:
moviesDf = moviesDf.withColumn("current_epoch_timestamp", unix_timestamp(current_timestamp()))
display(moviesDf)

In [0]:
moviesDf = moviesDf.withColumn("years_elapsed", lit(year(current_timestamp().cast("date")) - col("year")))

display(moviesDf)

In [0]:
from pyspark.sql.types import IntegerType
moviesDf = moviesDf.withColumn("budget", split(moviesDf["budget"], " ")[1].cast(IntegerType()))

display(moviesDf)

In [0]:
moviesDf = moviesDf.dropna()

display(moviesDf)

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
ratingsDf = spark.read.format("csv"). \
                option("header","true"). \
                option("inferSchema","true"). \
                load(filePath)

display(ratingsDf)

In [0]:
ratingsDf = ratingsDf.withColumn("current_epoch_timestamp", unix_timestamp(current_timestamp()))

display(ratingsDf)

In [0]:
filtered_males_avg_col = ratingsDf.filter((col("males_allages_avg_vote").isNotNull()))
mean_male = filtered_males_avg_col.select(avg(col("males_allages_avg_vote")).alias("mean_male")).collect()[0]["mean_male"]

filtered_females_avg_col = ratingsDf.filter((col("females_allages_avg_vote").isNotNull()))
mean_female = filtered_females_avg_col.select(avg(col("females_allages_avg_vote")).alias("femean_male")).collect()[0]["femean_male"]

if mean_male > mean_female:
  print("mezczyzni lepiej oceniaja")
else:
  print("kobiety lepiej oceniaja")

In [0]:
from pyspark.sql.types import LongType
ratingsDf = ratingsDf.dropna().withColumn("total_votes", col("total_votes").cast(LongType()))

ratingsDf.dtypes